# Final Project

In [1]:
!pip install mpi4py

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 18.7 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for mpi4py: filename=mpi4py-3.1.4-cp39-cp39-linux_x86_64.whl size=3380622 sha256=832345f065f8b4d8fff502d734373092d8685bdaabb4eb301984382e43c6d8bc
  Stored in directory: /root/.cache/pip/wheels/db/81/9f/43a031fce121c845baca1c5d9a1468cad98208286aa2832de9
Successfully built mpi4py


In [14]:
import os
arch = os.getenv("ARGS", "real")

In [15]:
try:
    import google.colab  # noqa: F401
except ImportError:
    import petsc4py
else:
    try:
        import petsc4py
    except ImportError:
        if arch != "complex":
            !wget "https://fem-on-colab.github.io/releases/petsc4py-install-real.sh" -O "/tmp/petsc4py-install.sh" && bash "/tmp/petsc4py-install.sh"
        else:
            !wget "https://fem-on-colab.github.io/releases/petsc4py-install-complex.sh" -O "/tmp/petsc4py-install.sh" && bash "/tmp/petsc4py-install.sh"
        import petsc4py

--2023-04-18 16:35:24--  https://fem-on-colab.github.io/releases/petsc4py-install-real.sh
Resolving fem-on-colab.github.io (fem-on-colab.github.io)... 185.199.108.153, 185.199.109.153, 185.199.110.153, ...
Connecting to fem-on-colab.github.io (fem-on-colab.github.io)|185.199.108.153|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1793 (1.8K) [application/x-sh]
Saving to: ‘/tmp/petsc4py-install.sh’

/tmp/petsc4py-insta 100%[===================>]   1.75K  --.-KB/s    in 0s      

2023-04-18 16:35:24 (18.8 MB/s) - ‘/tmp/petsc4py-install.sh’ saved [1793/1793]

+ INSTALL_PREFIX=/usr/local
++ echo /usr/local
++ awk -F/ '{print NF-1}'
+ INSTALL_PREFIX_DEPTH=2
+ PROJECT_NAME=fem-on-colab
+ SHARE_PREFIX=/usr/local/share/fem-on-colab
+ PETSC4PY_INSTALLED=/usr/local/share/fem-on-colab/petsc4py.installed
+ [[ ! -f /usr/local/share/fem-on-colab/petsc4py.installed ]]
+ H5PY_INSTALL_SCRIPT_PATH=https://github.com/fem-on-colab/fem-on-colab.github.io/raw/9d19467/releases/h5py-

# Part 1 : Answer 

In [1]:
import numpy as np
import scipy.sparse
from mpi4py import MPI

comm   = MPI.COMM_WORLD
nprocs = comm.Get_size()
rank   = comm.Get_rank()

### Class SparseMatrix

In [2]:
class SparseMatrix:

    def __init__(self, arg):
        '''
        Constructor for the SparseMatrix class.
        
        Parameters:
            arg (numpy.ndarray or tuple): The argument to initialize the SparseMatrix object.
            
        Raises:
            ValueError: If arg is a numpy array and its ndim is not equal to 2.
            TypeError: If arg is not a numpy array or a tuple of 3 numpy arrays.
            ValueError: If arg is a tuple of numpy arrays and the first two elements are not 1D arrays.
            ValueError: If the size of the input arrays in the tuple is not the same.
            
        '''
        if isinstance(arg, np.ndarray):
            if arg.ndim != 2:
                raise ValueError("arg should be a 2D numpy array")
            self.shape = arg.shape
            coo = scipy.sparse.coo_matrix(arg)
        elif isinstance(arg, tuple) and len(arg) == 3:
            x, Y, values = arg
            if not all(isinstance(a, np.ndarray) for a in [x, Y, values]):
                raise TypeError("arg should be a tuple of 3 numpy arrays")
            if not all(a.ndim == 1 for a in [x, Y]):
                raise ValueError("the first two elements of the tuple should be 1D numpy arrays")
            if x.size != values.size or Y.size != values.size:
                raise ValueError("the size of the 3 arrays should be the same")
            self.shape = (int(Y.max())+1, int(x.max())+1)
            coo = scipy.sparse.coo_matrix((values, (Y, x)), shape=self.shape)
        else:
            raise TypeError("arg should be a numpy array or a tuple of 3 numpy arrays")
        self.data = coo.data
        self.row = coo.row
        self.col = coo.col


    def cooTranspose(self):
        '''
        Transpose the SparseMatrix object in COO format.
        
        Returns:
            SparseMatrix: The transposed SparseMatrix object.
            
        '''
        transpose_coo = scipy.sparse.coo_matrix((self.data, (self.col, self.row)), shape=(self.shape[1], self.shape[0]))
        transpose_matrix = SparseMatrix((transpose_coo.row, transpose_coo.col, transpose_coo.data))
        transpose_matrix.row, transpose_matrix.col = transpose_matrix.col, transpose_matrix.row
        return transpose_matrix


    def cooMatVec(self, x):
        '''
        Multiply the SparseMatrix object with a vector x in COO format.
        
        Parameters:
            x (numpy.ndarray): The input vector to be multiplied with the SparseMatrix.
            
        Returns:
            numpy.ndarray: The resulting vector after the matrix-vector multiplication.
            
        Raises:
            ValueError: If the number of columns in the SparseMatrix does not match the size of the input vector.
            
        '''
        if self.shape[1] != x.shape[0]:
            raise ValueError("Matrix and vector dimensions are incompatible")
        y = np.zeros(self.shape[0])
        for i, j, v in zip(self.row, self.col, self.data):
            y[i] += v * x[j]
        return y



    def cooMatMat(self, B):
        '''
        Multiply the SparseMatrix object with another SparseMatrix B in COO format.
        
        Parameters:
            B (SparseMatrix): The input SparseMatrix to be multiplied with the current SparseMatrix.
            
        Returns:
            numpy.ndarray: The resulting matrix after the matrix-matrix multiplication.
            
        Raises:
            ValueError: If the number of columns in the current SparseMatrix does not match the number of rows in SparseMatrix B.
            
        '''
        if self.shape[1] != B.shape[0]:
            raise ValueError("Matrix dimensions are incompatible")
        C = np.zeros((self.shape[0], B.shape[1]))
        for i, j, v in zip(self.row, self.col, self.data):
            C[i] += v * B[j]
        return C

### Create a sparse matrix from a numpy array

In [3]:
A = np.array([[0, 0, 3, 0], [0, 0, 0, 0], [1, 2, 0, 0], [0, 0, 0, 4]])
M = SparseMatrix(A)
print(M.data)   
print(M.row)    
print(M.col)    
print(M.shape)  

[3 1 2 4]
[0 2 2 3]
[2 0 1 3]
(4, 4)


### Test transpose method

In [4]:
M_T = M.cooTranspose()
print(M_T.data)   
print(M_T.row)    
print(M_T.col)    
print(M_T.shape)  

[3 1 2 4]
[2 0 1 3]
[0 2 2 3]
(4, 4)


### Test matrix-vector multiplication

In [5]:
x = np.array([1, 2, 3, 4])
y = M.cooMatVec(x)
print(y)  

[ 9.  0.  5. 16.]


In [6]:
A@x

array([ 9,  0,  5, 16])

### Test matrix-matrix multiplication

In [7]:
B = np.array([[1, 0, 0], [0, 1, 0], [0, 0, 1], [0, 0, 0]])
C = M.cooMatMat(B)
print(C)  

[[0. 0. 3.]
 [0. 0. 0.]
 [1. 2. 0.]
 [0. 0. 0.]]


In [8]:
A@B

array([[0, 0, 3],
       [0, 0, 0],
       [1, 2, 0],
       [0, 0, 0]])

# Part 2 : Answer 

In [9]:
#Create the matrix and Rhs
np.random.seed(42)
from scipy.sparse import random
if rank == 0:
    n = 100
    # Set parameters for the sparse matrix
    density = 0.3 # density of non-zero elements (between 0 and 1)
    A =  random(n, n, density=density, format='csr')
    x = np.random.rand(n)
    B_all = np.dot(A.toarray(),x)
else:
    A = None
    B_all = None

In [10]:
def CreateLocalMatVec(A, B_all):
    if rank == 0:
        shape = A.shape
        nrows = shape[0]
        # split the number of rows evenly (as possible) among the MPI tasks
        N_pertask, extra = divmod(nrows, nprocs)
        # count: the size of each sub-task
        count = np.zeros(nprocs, dtype=int)
        count[:nprocs-extra] = N_pertask
        count[nprocs-extra:] = N_pertask+1
        # displacement: the starting index of each sub-task
        displ = np.zeros(nprocs, dtype=int)
        displ[1:] = np.cumsum(count)[:-1]
        #---- Send the relevant subsets of A and B to each slave MPI task ----
        for i in range(1,nprocs):
            # Get the start and end row index for this MPI task
            rstart = displ[i]
            rend   = rstart + count[i]
            #---- Get the subsets of A and B using these rows ----
            A_indptr  = A.indptr[rstart:rend+1]-A.indptr[rstart]
            pstart    = A.indptr[rstart]
            pend      = A.indptr[rend]
            A_indices = A.indices[pstart:pend]
            A_data    = A.data[pstart:pend]
            B         = B_all[rstart:rend]
            # Save the lengths of each array
            lengths = {
                    'A_indptr' : A_indptr.shape,
                    'A_indices': A_indices.shape,
                    'A_data'   : A_data.shape,
                    'B'        : B.shape
                    }
            # Send the arrays and their lengths to the relevant MPI task
            comm.send(lengths, dest=i)
            comm.Send(A_indptr,  dest=i)
            comm.Send(A_indices, dest=i)
            comm.Send(A_data,    dest=i)
            comm.Send(B,         dest=i)
        #---- Set the relevant subsets of A and B for the master MPI task (we don't need to do an MPI Send)
        rstart = displ[0]
        rend   = rstart + count[0]
        A_indptr  = A.indptr[rstart:rend+1]-A.indptr[rstart]
        pstart    = A.indptr[rstart]
        pend      = A.indptr[rend]
        A_indices = A.indices[pstart:pend]
        A_data    = A.data[pstart:pend]
        B         = B_all[rstart:rend]
    else:
        # Receive the array lengths
        lengths   = comm.recv(source=0)
        # Initialise the buffers
        A_indptr  = np.empty(lengths['A_indptr'], dtype=int)
        A_indices = np.empty(lengths['A_indices'], dtype=int)
        A_data    = np.empty(lengths['A_data'], dtype=float)
        B         = np.empty(lengths['B'], dtype=float)
        # Receive the arrays
        comm.Recv(A_indptr,  source=0)
        comm.Recv(A_indices, source=0)
        comm.Recv(A_data,    source=0)
        comm.Recv(B,         source=0)
        shape = None
    #broadcast shape
    shape = comm.bcast(shape, root=0)
    return (A_indptr, A_indices, A_data, B, shape)

In [11]:
class LinearSystem():
    def __init__(self, A=None, shape=None, rhs=None, solver=None, comm=None):
        
        from petsc4py import PETSc
        self.opts = PETSc.Options()

        self.ksp = PETSc.KSP()
        self.ksp.create()
        
        mat = PETSc.Mat().createAIJ(comm=comm, size=shape, csr=A)
        mat.setUp()
        mat.assemblyBegin()
        mat.assemblyEnd()
        self.sol, self.rhs = mat.getVecs()
        self.rhs.setArray(rhs)
        
        
        #---- Set up solver -----
        self.ksp = PETSc.KSP().create(comm=comm)
        
        # It is commonly used with the direct solver preconditioners like PCLU and PCCHOLESKY
        self.ksp.setType('preonly')
        pc = self.ksp.getPC()
        pc.setType('lu')
        pc.setFactorSolverType(solver)
        
        self.ksp.setOperators(mat)
        self.ksp.setFromOptions() # Apply any command line options
        self.ksp.setUp()
    
    def solve(self): 
        # st=timeit.default_timer()
        self.ksp.solve(self.rhs, self.sol)

In [16]:
(A_indptr, A_indices, A_data, b, shape) = CreateLocalMatVec(A, B_all)

L = LinearSystem(A=(A_indptr, A_indices, A_data), rhs=b, shape=shape, solver="mumps")
ts = MPI.Wtime()
L.solve()
te = MPI.Wtime()
tt = comm.reduce(te - ts, op=MPI.MAX, root=0)
if rank == 0:
    print("Timing to solve the linear system with petsc", tt)

Timing to solve the linear system with petsc 0.0008485709999999994


In [17]:
#---- Gather the solution onto a single array on the master MPI task
if rank == 0:
    X = np.empty(shape[0],dtype=np.double)
else:
    X = None
comm.Gatherv(L.sol.array,X)

In [18]:
np.random.seed(42)

if rank == 0:
    B = SparseMatrix(A.toarray())
    
    from scipy import sparse
    from scipy.sparse.linalg import spsolve
    mat = sparse.csr_matrix((B.data, (B.row, B.col)))
    ts = MPI.Wtime()
    sol = spsolve(mat, b)
    te = MPI.Wtime()
    print("Timing to solve the linear system with spsolve", te - ts)

Timing to solve the linear system with spsolve 0.001681607000000085


In [19]:
if rank == 0:
    print(np.allclose(X,sol))

True


# Interpretation

The timing results indicate that solving the linear system with PETSc (petsc) took 0.0008485709999999994 seconds, while solving the linear system with spsolve took 0.001681607000000085 seconds.

Based on these results, it appears that solving the linear system with PETSc is faster than using spsolve.

The discrepancy in timing could be due to several factors, including the efficiency of the underlying algorithms, implementation details, and the specific characteristics of the linear system being solved. PETSc is a high-performance parallel numerical library for solving large-scale linear and nonlinear systems of equations, and it is known for its efficiency and scalability. On the other hand, spsolve is a function from the scipy library, which provides a general-purpose sparse linear system solver, but it may not be optimized for performance in all scenarios.